In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.7.3/en_core_web_trf-3.7.3.tar.gz langchain-community faiss-gpu-cu11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.3 MB/s eta 0:00:000:00:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 37.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 99.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import spacy
import torch

In [4]:
torch.set_grad_enabled(False)
spacy.require_gpu()
nlp = spacy.load("en_core_web_trf")

In [5]:
def extract_propositions(text):
    doc = nlp(text)
    propositions = []
    for sent in doc.sents:
        clause = []
        for token in sent:
            clause.append(token.text)
            if token.text in [",", "and", "or", ";"] or token.dep_ in ["cc", "conj"]:
                propositions.append(" ".join(clause).strip())
                clause = []
        if clause:
            propositions.append(" ".join(clause).strip())
    return [p for p in propositions if len(p.split()) >= 3]

In [6]:
from datasets import load_dataset

In [7]:
dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus", split="passages")

README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['passage', 'id'],
    num_rows: 3200
})

In [9]:
passages = [item["passage"] for item in dataset if item["passage"]]

In [10]:
passages[:10]

['Uruguay (official full name in  ; pron.  , Eastern Republic of  Uruguay) is a country located in the southeastern part of South America.  It is home to 3.3 million people, of which 1.7 million live in the capital Montevideo and its metropolitan area.',
 'It is bordered by Brazil to the north, by Argentina across the bank of both the Uruguay River to the west and the estuary of RÃ\xado de la Plata to the southwest, and the South Atlantic Ocean to the southeast. It is the second smallest independent country in South America, larger only than Suriname and the French overseas department of French Guiana.',
 'Montevideo was founded by the Spanish in the early 18th century as a military stronghold. Uruguay won its independence in 1828 following a three-way struggle between Spain, Argentina and Brazil. It is a constitutional democracy, where the president fulfills the roles of both head of state and head of government',
 'The economy is largely based in agriculture (making up 10% of the GDP

In [11]:
from tqdm.notebook import tqdm

In [12]:
all_chunks = []
for passage in tqdm(passages, desc="Splitting into propositions"):
    all_chunks.extend(extract_propositions(passage))

Splitting into propositions:   0%|          | 0/3200 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


In [13]:
all_chunks[:10]

['Uruguay ( official full name in   ;',
 'pron .   ,',
 'Eastern Republic of   Uruguay ) is a country located in the southeastern part of South America .',
 'It is home to 3.3 million people ,',
 'of which 1.7 million live in the capital Montevideo and',
 'its metropolitan area',
 'It is bordered by Brazil to the north ,',
 'by Argentina across the bank of both the Uruguay River to the west and',
 'of RÃ\xado de la Plata to the southwest ,',
 'the South Atlantic Ocean']

In [14]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [15]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_35/412152783.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
2025-05-31 14:20:29.847356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748701230.055788      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748701230.114935      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
documents = [Document(page_content=chunk) for chunk in all_chunks]

In [17]:
documents[:10]

[Document(metadata={}, page_content='Uruguay ( official full name in   ;'),
 Document(metadata={}, page_content='pron .   ,'),
 Document(metadata={}, page_content='Eastern Republic of   Uruguay ) is a country located in the southeastern part of South America .'),
 Document(metadata={}, page_content='It is home to 3.3 million people ,'),
 Document(metadata={}, page_content='of which 1.7 million live in the capital Montevideo and'),
 Document(metadata={}, page_content='its metropolitan area'),
 Document(metadata={}, page_content='It is bordered by Brazil to the north ,'),
 Document(metadata={}, page_content='by Argentina across the bank of both the Uruguay River to the west and'),
 Document(metadata={}, page_content='of RÃ\xado de la Plata to the southwest ,'),
 Document(metadata={}, page_content='the South Atlantic Ocean')]

In [18]:
vectordb = FAISS.from_documents(documents, embedding_model)
vectordb.save_local("faiss_index")

In [19]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [20]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [21]:
from huggingface_hub import login
login(token="<token>")

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [24]:
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

Device set to use cuda:0


In [28]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnableMap

In [29]:
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the context to answer questions."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
    ("system", "Context:\n{context}\nAnswer:")
])

In [32]:
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

/tmp/ipykernel_35/2844409624.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [33]:
rag_chain = (
    RunnableMap({
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],
        "chat_history": lambda _: memory.load_memory_variables({})["chat_history"]
    }) |
    prompt |
    llm
)

In [34]:
question = "What is the capital of France?"
response = rag_chain.invoke({"question": question})
print(response)
memory.chat_memory.add_user_message(question)
memory.chat_memory.add_ai_message(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
System: Context:
[Document(id='cfffea6b-337a-4d7e-99a3-a1eb25220386', metadata={}, page_content='capital of the country .'), Document(id='5a5e127e-0bda-4281-8d6a-1d0f6ea9e2fa', metadata={}, page_content='The French government ,'), Document(id='850f3cf3-8afe-4e07-a4d5-9c1760b42011', metadata={}, page_content='( French - Speaking Countries ) .'), Document(id='eba3affa-767e-42a8-a98d-a4ec9f761708', metadata={}, page_content='twice the size of France ,'), Document(id='e8f44628-692a-406d-87d1-bc02464546c5', metadata={}, page_content='the capital ,')]
Answer: The capital of France is Paris.


In [35]:
question = "How was capital of France decided?"
response = rag_chain.invoke({"question": question})
print(response)
memory.chat_memory.add_user_message(question)
memory.chat_memory.add_ai_message(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
AI: System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
System: Context:
[Document(id='cfffea6b-337a-4d7e-99a3-a1eb25220386', metadata={}, page_content='capital of the country .'), Document(id='5a5e127e-0bda-4281-8d6a-1d0f6ea9e2fa', metadata={}, page_content='The French government ,'), Document(id='850f3cf3-8afe-4e07-a4d5-9c1760b42011', metadata={}, page_content='( French - Speaking Countries ) .'), Document(id='eba3affa-767e-42a8-a98d-a4ec9f761708', metadata={}, page_content='twice the size of France ,'), Document(id='e8f44628-692a-406d-87d1-bc02464546c5', metadata={}, page_content='the capital ,')]
Answer: The capital of France is Paris.
Human: How was capital of France decided?
System: Context:
[Document(id='5a5e127e-0bda-4281-8d6a-1d0f6ea9e2fa', metadata={}, page_content='The French government ,'), Document(id='e

In [36]:
question = "Who discovered gravity?"
response = rag_chain.invoke({"question": question})
print(response)
memory.chat_memory.add_user_message(question)
memory.chat_memory.add_ai_message(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
AI: System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
System: Context:
[Document(id='cfffea6b-337a-4d7e-99a3-a1eb25220386', metadata={}, page_content='capital of the country .'), Document(id='5a5e127e-0bda-4281-8d6a-1d0f6ea9e2fa', metadata={}, page_content='The French government ,'), Document(id='850f3cf3-8afe-4e07-a4d5-9c1760b42011', metadata={}, page_content='( French - Speaking Countries ) .'), Document(id='eba3affa-767e-42a8-a98d-a4ec9f761708', metadata={}, page_content='twice the size of France ,'), Document(id='e8f44628-692a-406d-87d1-bc02464546c5', metadata={}, page_content='the capital ,')]
Answer: The capital of France is Paris.
Human: How was capital of France decided?
AI: System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
AI: System: You are a 

In [37]:
question = "How was it discovered?"
response = rag_chain.invoke({"question": question})
print(response)
memory.chat_memory.add_user_message(question)
memory.chat_memory.add_ai_message(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
AI: System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
System: Context:
[Document(id='cfffea6b-337a-4d7e-99a3-a1eb25220386', metadata={}, page_content='capital of the country .'), Document(id='5a5e127e-0bda-4281-8d6a-1d0f6ea9e2fa', metadata={}, page_content='The French government ,'), Document(id='850f3cf3-8afe-4e07-a4d5-9c1760b42011', metadata={}, page_content='( French - Speaking Countries ) .'), Document(id='eba3affa-767e-42a8-a98d-a4ec9f761708', metadata={}, page_content='twice the size of France ,'), Document(id='e8f44628-692a-406d-87d1-bc02464546c5', metadata={}, page_content='the capital ,')]
Answer: The capital of France is Paris.
Human: How was capital of France decided?
AI: System: You are a helpful assistant. Use the context to answer questions.
Human: What is the capital of France?
AI: System: You are a 